# Z3 Solver Project Demo
Presentation purposes

*Svolto per la presentazione del progetto all'orale*

In [3]:
import pandas as pd 
import numpy as np 
import itertools 
from model import TimetableScheduler
from z3 import *

## Scenario 1: Toy Model
Simple model containing just a few sessions and courses, not intended to be realistic in any way

*Modello semplice che contiene qualche sessione e corsi, inteso solo per motivi di testing*

In [2]:
toy_schedule = TimetableScheduler('sample_1', 8)

In [3]:
toy_schedule.start()

In [4]:
toy_schedule.add_constraints()

In [5]:
toy_schedule.solve()

TIME TABLE SUCCESSFULLY CREATED


In [7]:
timetable = []

In [8]:
names = toy_schedule.database.get_names()

In [9]:

for t in toy_schedule.T:
    t_slots = []

    for (S, A) in itertools.product(toy_schedule.indexes['Sessions'], toy_schedule.indexes['Rooms']):
        S_c = toy_schedule.database.get_course(S)
        if toy_schedule.model.evaluate(toy_schedule.X[S,t,A], model_completion=True):
            t_slots.append(f"{names['Courses'][toy_schedule.database.get_course(S)]} [{names['Rooms'][A]}]")
                
    if t_slots == []:
        t_slots = ["NESSUNA LEZIONE"]
    timetable.append("\n".join(t_slots))

In [10]:
timetable

['ANALISI 1 [Aula Ciamician]',
 'ANALISI 1 [Aula Ciamician]',
 'ANALISI 1 [Aula Ciamician]',
 'GEOMETRIA [Aula Ciamician]',
 'GEOMETRIA [Aula Ciamician]',
 'ANALISI 1 [Aula Ciamician]',
 'ANALISI 1 [Aula Ciamician]',
 'NESSUNA LEZIONE',
 'ANALISI 2 [Aula Morin 5]',
 'ANALISI 2 [Aula Morin 5]',
 'ANALISI 2 [Aula Morin 5]',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'NESSUNA LEZIONE',
 'NESSUNA LEZIONE',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'PROGRAMMAZIONE [Aula Ciamician]',
 'SISTEMI OPERATIVI [Aula Morin]',
 'SISTEMI OPE

In [11]:
rows = ["08:00-09:00","09:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00"]
columns = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]

In [12]:
np_timetable = np.array(timetable, dtype=object)
np_timetable = np_timetable.reshape(6,8)

In [13]:
df = pd.DataFrame(np_timetable, columns=rows, index=columns)

In [14]:
from IPython.display import display, HTML # per fare una presentazione grafica decente
display(HTML(df.T.to_html().replace("\\n", "<br>")))

,Lunedì,Martedì,Mercoledì,Giovedì,Venerdì,Sabato
08:00-09:00,ANALISI 1 [Aula Ciamician],ANALISI 2 [Aula Morin 5],NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE
09:00-10:00,ANALISI 1 [Aula Ciamician],ANALISI 2 [Aula Morin 5],NESSUNA LEZIONE,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],SISTEMI OPERATIVI [Aula Morin 6]
10:00-11:00,ANALISI 1 [Aula Ciamician],ANALISI 2 [Aula Morin 5],NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],PROGRAMMAZIONE [Aula Ciamician],SISTEMI OPERATIVI [Aula Morin 6]
11:00-12:00,GEOMETRIA [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],PROGRAMMAZIONE [Aula Ciamician],NESSUNA LEZIONE
12:00-13:00,GEOMETRIA [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],SISTEMI OPERATIVI [Aula Morin],SISTEMI OPERATIVI [Aula Morin 2]GEOMETRIA [Aula Morin]
13:00-14:00,ANALISI 1 [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],SISTEMI OPERATIVI [Aula Morin],SISTEMI OPERATIVI [Aula Morin 2]GEOMETRIA [Aula Morin]
14:00-15:00,ANALISI 1 [Aula Ciamician],NESSUNA LEZIONE,NESSUNA LEZIONE,PROGRAMMAZIONE [Aula Ciamician],NESSUNA LEZIONE,GEOMETRIA [Aula Morin]
15:00-16:00,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE,NESSUNA LEZIONE


In [16]:
len(toy_schedule.solver.assertions())

19830

## Scenario 2: Next Semester's Courses for AIDA
Simplified model containing (estimated) sessions and courses for the next semester in AIDA.

*Modello semplificato che contiene i corsi del prossimo semestre del CdS triennale in IADA (2025-2026, Primo Semestre)*

Vincoli aggiuntivi: 
- Non ci sono lezioni il sabato
$$
\forall S, T, A, \text{ s.t. }T\text{ rappresenta un'ora in sabato }(\lfloor T/6 \rfloor = 6), X_{S,T,A} \implies \text{False}
$$

- (Forse) Gli studenti del 1° anno e del 2°, 3° anno hanno orari di pranzo diversi per evitare il sovraffollamento delle mense (2°, 3° sono raggruppati assieme per semplicità)
    - Dopo un sondaggio si è deciso che quelli del 1° anno pranzano alle 12-13 e quelli del 2°, 3° anno alle 13-14

In [17]:
orario_aida = TimetableScheduler('aida', 9)

In [18]:
orario_aida.start()

In [19]:
from z3 import Not

In [20]:
# add custom constraint
for (S,T,A) in itertools.product(orario_aida.indexes['Sessions'],
                                 orario_aida.T,
                                 orario_aida.indexes['Rooms']):
    if (T // 9) == (6-1):
        orario_aida.solver.add(Not(orario_aida.X[S,T,A]))


In [22]:
orario_aida.add_constraints()

In [23]:
len(orario_aida.solver.assertions())

67394

In [24]:

orario_aida.solve()

TIME TABLE SUCCESSFULLY CREATED


In [26]:
# stampa l'orario per AIDA
names = orario_aida.database.get_names()
aida_timeslots = []

for t in orario_aida.T:
    t_slots = []

    for (S, A) in itertools.product(orario_aida.indexes['Sessions'], orario_aida.indexes['Rooms']):
        S_c = orario_aida.database.get_course(S)
        if orario_aida.model.evaluate(orario_aida.X[S,t,A], model_completion=True):
            t_slots.append(f"{names['Courses'][orario_aida.database.get_course(S)]} [{names['Rooms'][A]}]")
                
    if t_slots == []:
        t_slots = ['NESSUNA LEZIONE']
    aida_timeslots.append(" > " + "\n > ".join(t_slots))

In [27]:
rows = ["09:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00","16:00-17:00","17:00-18:00"]
columns = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]

np_aida_timetable = np.array(aida_timeslots, dtype=object)
np_aida_timetable = np_aida_timetable.reshape(6,9)
df = pd.DataFrame(np_aida_timetable, columns=rows, index=columns)

In [28]:
display(HTML(df.T.to_html().replace("\\n", "<br>")))

,Lunedì,Martedì,Mercoledì,Giovedì,Venerdì,Sabato
09:00-10:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5C]","> ANALISI 1 [H3, 1A] > PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> SISTEMI COMPLESSI [H3, 1B]","> ALGORITMI E STRUTTURE DATI [H2 Bis, 2A MORIN] > SISTEMI COMPLESSI [H2 Bis, 3B]","> COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5B]",> NESSUNA LEZIONE
10:00-11:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5C]","> ANALISI 1 [H3, 1A] > PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> INFERENZA STATISTICA [H3, 1C] > SISTEMI COMPLESSI [H3, 1B]","> ALGORITMI E STRUTTURE DATI [H2 Bis, 2A MORIN] > SISTEMI COMPLESSI [H2 Bis, 3B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5B]",> NESSUNA LEZIONE
11:00-12:00,"> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1A]","> ARCHITETTURE DEGLI ELABORATORI [H3, 2B]","> INFERENZA STATISTICA [H3, 1C] > SISTEMI COMPLESSI [H3, 1B]","> ANALISI 1 [H3, 1B] > INFERENZA STATISTICA [H2 Bis, 2A MORIN]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H2 Bis, 5B]",> NESSUNA LEZIONE
12:00-13:00,> NESSUNA LEZIONE,"> ARCHITETTURE DEGLI ELABORATORI [H3, 2B]","> PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> ANALISI 1 [H3, 1B] > INFERENZA STATISTICA [H2 Bis, 2A MORIN]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C]",> NESSUNA LEZIONE
13:00-14:00,"> PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 1B]","> ARCHITETTURE DEGLI ELABORATORI [H3, 2B]","> PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 2A]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 5A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 1B] > COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C]",> NESSUNA LEZIONE
14:00-15:00,"> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 1A] > PROGRAMMAZIONE AVANZATA E PARALLELA [H3, 1B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B]",> NESSUNA LEZIONE,"> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H2 Bis, 2A MORIN] > INTRODUZIONE AL MACHINE LEARNING [H2 Bis, 5A]","> COMPUTABILITà, COMPLESSITà E LOGICA [H3, 2C]",> NESSUNA LEZIONE
15:00-16:00,"> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 1A] > ALGORITMI E STRUTTURE DATI [H2 Bis, 3B] > INTRODUZIONE AL MACHINE LEARNING [H3, 1B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H3, 2A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B]","> ANALISI 1 [H3, 2B] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H3, 1B]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 1A] > INFERENZA STATISTICA [H3, 1C] > INTRODUZIONE AL MACHINE LEARNING [H3, 2C]",> NESSUNA LEZIONE
16:00-17:00,"> ALGORITMI E STRUTTURE DATI [H2 Bis, 3B] > INTRODUZIONE AL MACHINE LEARNING [H3, 1B]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B] > ALGORITMI E STRUTTURE DATI [H3, 2A]","> INTRODUZIONE ALLA PROGRAMMAZIONE E LABORATORIO [H3, 2B]","> ANALISI 1 [H3, 2B] > INTRODUZIONE ALL INTELLIGENZA ARTIFICIALE [H3, 1B]","> ALGEBRA LINEARE ED ELEMENTI DI GEOMETRIA [H3, 1A] > INFERENZA STATISTICA [H3, 1C] > INTRODUZIONE AL MACHINE LEARNING [H3, 2C]",> NESSUNA LEZIONE
17:00-18:00,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE,> NESSUNA LEZIONE


In [29]:
df.to_excel("./timetables/aida.xlsx")

In [33]:
len(orario_aida.X)

20088

## Scenario 3: High School Timetable
Model creating a time table for the lessons of a class in high school, the last 3 years

*Modello che crea un orario settimanale per le lezioni del trienno di un liceo linguistico, solo per una classe (per semplicità assumiamo che tutti i professori siano assegnati ad una e sola classe)*

Richieste specifiche:
- Nel quinto anno, le professoresse di Italiano e delle materie d'indirizzo (i.e. Inglese, Tedesco e Russo) hanno chiesto di ottenere uno slot da 3 ore, in un modo tale da consentire di svolgere delle esercitazioni della prima e seconda prova in vista della maturità.
- Inoltre, la professoressa di Storia e Filosofia ha chiesto di avere almeno uno slot da due ore, per delle motivazioni analoghe della prof. di Italiano.
- Tutte le classi hanno richiesto di avere le due ore di motoria attaccate, per evitare scomodità pratiche. 
- In vista agli esami di certificazioni delle lingue straniere Inglese e Tedesco, i professori hanno richiesto di ottenere almeno un slot da due ore per svolgere delle esercitazioni.

Vincoli specifici:
- Tutte le lezioni, tranne quelle di Motoria, saranno assegnate all'aula della classe. Ad esempio, le lezioni di Italiano del 3° anno verranno assegnate solamente all'aula dedicata agli alunni del 3° anno.
- Esiste solo un'aula per svolgere le lezioni di Motoria e quindi verrà condivisa tra le classi

In [1]:
import pandas as pd 
import numpy as np 
import itertools 
from model import TimetableScheduler
from z3 import *

In [2]:
liceo_timetable = TimetableScheduler('liceo', 5)

# Ottimizzazione: ho cambiato la "tattica" del solver per favorire le propagazioni, risolvendo il problema più velocemene
tac = Then('propagate-values','sat')
liceo_timetable.solver = tac.solver()

In [3]:
liceo_timetable.start()

In [4]:
liceo_timetable.add_constraints()

In [5]:
# Vincoli sulle aule

classi = liceo_timetable.indexes['CdS']
for classe in classi:
    corsi = liceo_timetable.database.get_courses(classe)
    for corso in corsi: 
        if corso in [1501, 2501, 3501]:
            continue # esclude motoria

        sessioni = liceo_timetable.database.get_sessions(corso)
        liceo_timetable.solver.add(
            And(
                [ Not(liceo_timetable.X[S, T, A]) for (S,T,A) in itertools.product(sessioni, liceo_timetable.T, liceo_timetable.indexes['Rooms']) if A != classe]
            
            )
        )
        

In [6]:
# Vincolo sulla palestra
sessioni_motoria = [
    i[0] for i in liceo_timetable.database.execute_query(
        """
            SELECT DISTINCT IDSession
            FROM Session
            WHERE IDCourse = 1501 OR IDCourse = 2501 OR IDCourse = 3501; 
        """
    )
]

liceo_timetable.solver.add(
    And(
        [Not(liceo_timetable.X[S, T, A]) for (S,T,A) in itertools.product(sessioni_motoria, liceo_timetable.T, liceo_timetable.indexes['Rooms']) if A in [1,2,3]]
    )
)

In [10]:
liceo_timetable.solve()

TIME TABLE SUCCESSFULLY CREATED


In [ ]:
# stampa l'orario per il liceo
names = liceo_timetable.database.get_names()
liceo_timeslots = []

for t in liceo_timetable.T:
    t_slots = []

    for (S, A) in itertools.product(liceo_timetable.indexes['Sessions'], liceo_timetable.indexes['Rooms']):
        S_c = liceo_timetable.database.get_course(S)
        if liceo_timetable.model.evaluate(liceo_timetable.X[S,t,A], model_completion=True):
            t_slots.append(f"{names['Courses'][liceo_timetable.database.get_course(S)]} [{names['Rooms'][A]}]")
                
    if t_slots == []:
        t_slots = ['NESSUNA LEZIONE']
    liceo_timeslots.append(" > " + "\n > ".join(t_slots))

In [18]:
liceo_timeslots

[" > SCIENZE NATURALI [ANNO III]\n > RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO IV]\n > TERZA LINGUA STRANIERA (RUSSO) [ANNO V]",
 " > RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO III]\n > SCIENZE E MOTORIE SPORTIVE (ANNO II) [PALESTRA]\n > LINGUA E LETTERATURA ITALIANA [ANNO V]",
 " > FISICA [ANNO III]\n > SCIENZE E MOTORIE SPORTIVE (ANNO II) [PALESTRA]\n > RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO V]",
 " > FISICA [ANNO III]\n > STORIA DELL'ARTE [ANNO IV]\n > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO V]",
 " > MATEMATICA [ANNO III]\n > STORIA DELL'ARTE [ANNO IV]\n > FILOSOFIA [ANNO V]",
 ' > SCIENZE E MOTORIE SPORTIVE (ANNO I) [PALESTRA]\n > FISICA [ANNO IV]\n > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V]',
 ' > SCIENZE E MOTORIE SPORTIVE (ANNO I) [PALESTRA]\n > MATEMATICA [ANNO IV]\n > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V]',
 ' > LINGUA E LETTERATURA ITALIANA [ANNO III]\n > FISICA [ANNO IV]\n > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V]',
 ' > LINGUA E LETTERATURA I

In [19]:
rows = ["08:00-09:00","09:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00"]
columns = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato"]

np_liceo_timetable = np.array(liceo_timeslots, dtype=object)
np_liceo_timetable = np_liceo_timetable.reshape(6, 5)
df = pd.DataFrame(np_liceo_timetable, columns=rows, index=columns)

In [ ]:
from IPython.display import display, HTML 
display(HTML(df.T.to_html().replace("\\n", "<br>")))

,Lunedì,Martedì,Mercoledì,Giovedì,Venerdì,Sabato
08:00-09:00,> SCIENZE NATURALI [ANNO III] > RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO IV] > TERZA LINGUA STRANIERA (RUSSO) [ANNO V],> SCIENZE E MOTORIE SPORTIVE (ANNO I) [PALESTRA] > FISICA [ANNO IV] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V],> SECONDA LINGUA STRANIERA (TEDESCO) [ANNO III] > TERZA LINGUA STRANIERA (RUSSO) [ANNO IV] > STORIA [ANNO V],> LINGUA E LETTERATURA ITALIANA [ANNO III] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO IV] > SCIENZE E MOTORIE SPORTIVE (ANNO III) [PALESTRA],> TERZA LINGUA STRANIERA (RUSSO) [ANNO III] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO IV] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO V],> PRIMA LINGUA STRANIERA (INGLESE) [ANNO III] > LINGUA E LETTERATURA ITALIANA [ANNO IV] > MATEMATICA [ANNO V]
09:00-10:00,> RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO III] > SCIENZE E MOTORIE SPORTIVE (ANNO II) [PALESTRA] > LINGUA E LETTERATURA ITALIANA [ANNO V],> SCIENZE E MOTORIE SPORTIVE (ANNO I) [PALESTRA] > MATEMATICA [ANNO IV] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V],> STORIA DELL'ARTE [ANNO III] > TERZA LINGUA STRANIERA (RUSSO) [ANNO IV] > STORIA [ANNO V],> LINGUA E LETTERATURA ITALIANA [ANNO III] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO IV] > SCIENZE E MOTORIE SPORTIVE (ANNO III) [PALESTRA],> SCIENZE NATURALI [ANNO III] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO IV] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO V],> SECONDA LINGUA STRANIERA (TEDESCO) [ANNO III] > LINGUA E LETTERATURA ITALIANA [ANNO IV] > MATEMATICA [ANNO V]
10:00-11:00,> FISICA [ANNO III] > SCIENZE E MOTORIE SPORTIVE (ANNO II) [PALESTRA] > RELIGIONE CATTOCIA O ATTIVITA' ALTERNATIVE [ANNO V],> LINGUA E LETTERATURA ITALIANA [ANNO III] > FISICA [ANNO IV] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO V],> TERZA LINGUA STRANIERA (RUSSO) [ANNO III] > FILOSOFIA [ANNO IV] > LINGUA E LETTERATURA ITALIANA [ANNO V],> PRIMA LINGUA STRANIERA (INGLESE) [ANNO III] > SCIENZE NATURALI [ANNO IV] > STORIA DELL'ARTE [ANNO V],> STORIA [ANNO III] > LINGUA E LETTERATURA ITALIANA [ANNO IV] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO V],> FILOSOFIA [ANNO III] > PRIMA LINGUA STRANIERA (INGLESE) [ANNO IV] > TERZA LINGUA STRANIERA (RUSSO) [ANNO V]
11:00-12:00,> FISICA [ANNO III] > STORIA DELL'ARTE [ANNO IV] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO V],> LINGUA E LETTERATURA ITALIANA [ANNO III] > MATEMATICA [ANNO IV] > FILOSOFIA [ANNO V],> TERZA LINGUA STRANIERA (RUSSO) [ANNO III] > STORIA [ANNO IV] > LINGUA E LETTERATURA ITALIANA [ANNO V],> PRIMA LINGUA STRANIERA (INGLESE) [ANNO III] > TERZA LINGUA STRANIERA (RUSSO) [ANNO IV] > SCIENZE NATURALI [ANNO V],> SECONDA LINGUA STRANIERA (TEDESCO) [ANNO III] > LINGUA E LETTERATURA ITALIANA [ANNO IV] > FISICA [ANNO V],> STORIA [ANNO III] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO IV] > TERZA LINGUA STRANIERA (RUSSO) [ANNO V]
12:00-13:00,> MATEMATICA [ANNO III] > STORIA DELL'ARTE [ANNO IV] > FILOSOFIA [ANNO V],> MATEMATICA [ANNO III] > FILOSOFIA [ANNO IV] > STORIA DELL'ARTE [ANNO V],> TERZA LINGUA STRANIERA (RUSSO) [ANNO III] > STORIA [ANNO IV] > LINGUA E LETTERATURA ITALIANA [ANNO V],> STORIA DELL'ARTE [ANNO III] > TERZA LINGUA STRANIERA (RUSSO) [ANNO IV] > SCIENZE NATURALI [ANNO V],> SECONDA LINGUA STRANIERA (TEDESCO) [ANNO III] > SCIENZE NATURALI [ANNO IV] > FISICA [ANNO V],> FILOSOFIA [ANNO III] > SECONDA LINGUA STRANIERA (TEDESCO) [ANNO IV] > TERZA LINGUA STRANIERA (RUSSO) [ANNO V]
